In [1]:
import numpy as np
import pandas as pd
import sys
import os
from random import shuffle
import torch
import torch.nn as nn
from models.gat import GATNet
from models.gat_gcn import GAT_GCN
from models.gcn import GCNNet
from models.ginconv import GINConvNet
from utils import *

In [2]:
def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat(
                (total_labels, data.y.view(-1, 1).cpu()), 0)
    return total_labels.numpy().flatten(), total_preds.numpy().flatten()


In [3]:
# datasets=[['davis', 'kiba'][0]]

In [4]:
modeling = [GINConvNet, GATNet, GAT_GCN, GCNNet][0]
model_st = modeling.__name__

In [5]:
cuda_name = "cuda:0"

In [6]:
TRAIN_BATCH_SIZE = 512
TEST_BATCH_SIZE = 512
LR = 0.0005
LOG_INTERVAL = 20
NUM_EPOCHS = 1000

In [7]:
dataset = 'davis'

In [8]:
train_data = TestbedDataset(root='data', dataset=dataset+'_train')
test_data = TestbedDataset(root='data', dataset=dataset+'_test')

Pre-processed data found: data/processed/davis_train.pt, loading ...
Pre-processed data found: data/processed/davis_test.pt, loading ...


In [9]:
# if training_validation.py is being used
train_size = int(0.8 * len(train_data))
valid_size = len(train_data) - train_size
train_data, valid_data = torch.utils.data.random_split(train_data, [train_size, valid_size])        


# make data PyTorch mini-batch processing ready
train_loader = DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=TEST_BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

In [10]:
# train_loader = DataLoader(
#             train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
# test_loader = DataLoader(
#     test_data, batch_size=TEST_BATCH_SIZE, shuffle=False)

In [11]:
device = torch.device( cuda_name if torch.cuda.is_available() else "cpu")
model = modeling().to(device)

In [12]:
model.load_state_dict(torch.load('./NEW_FOLD_RESULTS/model_GINConvNet_davis99.model'))

<All keys matched successfully>

In [13]:
G,P = predicting(model, device, test_loader)

Make prediction for 5010 samples...


In [14]:
# len(train_data)

In [15]:
ret = [rmse(G, P), mse(G, P), pearson(G, P), spearman(G, P), ci(G, P)]

In [16]:
ret

[0.6021559376702689,
 0.36259177,
 0.797512231043664,
 0.6666758781324954,
 0.8675362776522862]

In [ ]:
R2,CI,RMSE,MSE = ret[2], ret[4], ret[0], ret[1]

In [ ]:
print(R2,CI,RMSE,MSE)

In [ ]:
print(pearson(G,P),ci(G,P),rmse(G,P),mse(G,P))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.title('GraphDTA Davis FOLD00',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
# plt.locator_params(axis="x", nbins=15)
# plt.locator_params(axis="y", nbins=15)
plt.xlabel('ground truth',fontsize='20')
plt.ylabel('predictions', fontsize='20')
plt.plot(G,P,'.')
# plt.plot(predictions_combined,predictions_ligand_only,'.')
plt.plot([5,10],[5,10])
plt.grid()